# Transporte Quântico Mesoscópico

## Quantum Matter - Materials & Concepts Summer School 2021

### Sessão prática

Nesta sessão, vamos calcular a autoenergia de contactos semi-infinitas de usando diferentes métodos numéricos, e usar esses resultados para calcular a transmissão através sistema de um único nível. 

Para tornar o problema concreto vamos focar-nos num cadeia de tight-binding de uma orbital com hoppings a primeiros vizinhos. A vantagem deste modelo, é que várias quantidades podem ser calculadas analiticamente, podendo servir de padrão para os resultados numéricos. (Podes tentar contactos mais gerais)

Para resolver o problema usando Julia é conveninente carregar o pacote `LinearAlgebra`, para resolver problemas de algebra linear, e instalar e carregar uma biblioteca de plotting. Vamos usar o `PyPlot`.

In [5]:
using LinearAlgebra

In [4]:
] add PyPlot

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [6]:
using PyPlot

Matplotlib is building the font cache; this may take a moment.


## 1. Cálculo da estrutura de bandas de um sistema 1D

Consideremos um sistema 1D invariante de translação. Este pode ser escrito como:

$$
H_{\text{infinito}}=\left[\begin{array}{cccccc}
 & \ddots\\
\ddots & \ddots & \boldsymbol{v}\\
 & \boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 &  & \boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 &  &  & \boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  &  &  & \ddots
\end{array}\right].
$$

Para calcular a estrutura de bandas, devemos resolver o problema de valores próprios:

$$
\left(\boldsymbol{h}+e^{ik}\boldsymbol{v}+e^{-ik}\boldsymbol{v}^{\dagger}\right)\Psi_{k}=E_{k}\Psi_{k}.
$$

### Exercício 1.1.
Escreve uma função que dado as matrizes $\boldsymbol{h}$ e $\boldsymbol{v}$ calcule a estrutura de bandas 1D.

Por conveniência podemos guardar o Hamiltoniano numa estrutura:

In [9]:
struct SemiInfiniteLead{Th, Tv}
    h::Th
    v::Tv
end

mat(x::Number) = fill(x, 1, 1)
mat(A::AbstractMatrix) = A
SemiInfiniteLead(e::Number, t::Number) = SemiInfiniteLead(mat(e), mat(t))

dimh(lead::SemiInfiniteLead) = size(lead.h, 2)
;

Uma cadeia simples 1D com $\boldsymbol{h} = 0$ e $\boldsymbol{v}=-t$ pode ser definida como:

In [11]:
chain = SemiInfiniteLead(0.0, -1.0)
;

## 2. Cálculo recursivo da auto-energia

Consideremos agora um contacto semi-infitio, com Hamiltoniano dado por:

$$
H_{\text{semi}}=\left[\begin{array}{cccc}
\boldsymbol{h} & \boldsymbol{v}\\
\boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 & \boldsymbol{v}^\dagger & \ddots & \ddots\\
 & \ddots
\end{array}\right]
$$

Queremos calcular a auto-energia: $\Sigma^R = \boldsymbol{v}\boldsymbol{g}_{1,1}^R \boldsymbol{v}^\dagger$,

onde a função de Green é a solução de:

$$
\left[\begin{array}{cccc}
E-\boldsymbol{h} & -\boldsymbol{v}\\
-\boldsymbol{v}^{\dagger} & E-\boldsymbol{h} & -\boldsymbol{v}\\
 & -\boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  & \ddots
\end{array}\right]\left[\begin{array}{cccc}
\boldsymbol{g}_{1,1}^{R} & \boldsymbol{g}_{1,2}^{R} & \boldsymbol{g}_{1,3}^{R} & \cdots\\
\boldsymbol{g}_{2,1}^{R} & \boldsymbol{g}_{2,2}^{R} & \boldsymbol{g}_{2,3}^{R} & \cdots\\
\boldsymbol{g}_{3,1}^{R} & \boldsymbol{g}_{3,2}^{R} & \boldsymbol{g}_{3,3}^{R} & \cdots\\
\vdots & \vdots & \vdots & \ddots
\end{array}\right]=\left[\begin{array}{cccc}
\boldsymbol{1}\\
 & \boldsymbol{1}\\
 &  & \boldsymbol{1} & \ddots\\
 &  & \ddots & \ddots
\end{array}\right]
$$

O método recursivo baseia-se no facto de que uma cadeia semi-infinita pode ser vista como um sítio acoplado a uma cadeia semi-infinita. Tratando a hibridização entre o primeiro sítio e o resto da cadeia como uma perturbação, podemos então escrever as equações de Dyson:

$$
\begin{align*}
\boldsymbol{g}^{R} & =\tilde{\boldsymbol{g}}^{R}+\tilde{\boldsymbol{g}}^{R}\cdot\boldsymbol{V}\cdot\boldsymbol{g}^{R}\\
\boldsymbol{g}^{R} & =\tilde{\boldsymbol{g}}^{R}+\boldsymbol{g}^{R}\cdot\boldsymbol{V}\cdot\tilde{\boldsymbol{g}}^{R}
\end{align*}
$$

Onde:
$$
\tilde{\boldsymbol{g}}^{R}=\left[\begin{array}{cccc}
E-\boldsymbol{h} & \boldsymbol{0}\\
\boldsymbol{0} & E-\boldsymbol{h} & -\boldsymbol{v}\\
 & -\boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  & \ddots
\end{array}\right]^{-1}
$$

é a função de Green para o sistema em que o primeiro sítio está desconectado do resto do contacto e
$$
\boldsymbol{V}=\left[\begin{array}{cccc}
\boldsymbol{0} & \boldsymbol{v}\\
\boldsymbol{v}^{\dagger} & \boldsymbol{0} & \boldsymbol{0}\\
 & \boldsymbol{0} & \boldsymbol{0} & \ddots\\
 &  & \ddots & \ddots
\end{array}\right]
$$

liga o primeiro sítio ao resto do contacto.

A partir daqui obtemos as seguintes relações:
$$
\begin{align*}
\boldsymbol{g}_{1,1}^{R} & =\tilde{\boldsymbol{g}}_{1,1}^{R}+\tilde{\boldsymbol{g}}_{1,1}^{R}\cdot\boldsymbol{V}_{1,2}\cdot\boldsymbol{g}_{2,1}^{R}\\
\boldsymbol{g}_{2,1}^{R} & =\tilde{\boldsymbol{g}}_{2,2}^{R}\cdot\boldsymbol{V}_{2,1}\cdot\boldsymbol{g}_{1,1}^{R}
\end{align*}
$$
De onde obtemos a equação:
$$
\boldsymbol{g}_{1,1}^{R}=\left[E-\boldsymbol{h}-\boldsymbol{v}\cdot\tilde{\boldsymbol{g}}_{2,2}^{R}\cdot\boldsymbol{v}^{\dagger}\right]^{-1}
$$

Parece que não avançamos muito, porque não sabemos calcular $\tilde{\boldsymbol{g}}^R_{2,2}$. No entanto, $\tilde{\boldsymbol{g}}^R_{2,2}$ é exactamente a quantidade que queremos calcular: a função de Green de um contacto semi-infinito, avaliada na fronteira do contacto. Logo, temos que $$\tilde{\boldsymbol{g}}^R_{2,2} = \boldsymbol{g}^R_{1,1}$$

e obtemos a seguinte equação auto-consistente:
$$
\boldsymbol{g}_{1,1}^{R}=\left[E-\boldsymbol{h}-\boldsymbol{v}\cdot\boldsymbol{g}_{1,1}^{R}\cdot\boldsymbol{v}^{\dagger}\right]^{-1}
$$

Que podemos usar para calcular a função de Green do contacto.

### Exercício 2.1. 
Para o caso partícular, de uma cadeia simples, reduza a equação anterior a uma equação quadrática para $g_{1,1}^R$. Das duas soluções possíveis, obtenha a função retardada de tal forma a que tomando uma energia complex $E + i \eta$, com $\eta \rightarrow 0^+$, a parte imaginária é negativa. 

Obtenha: 
$$
g^{R}(E)=\frac{1}{t}\left[\left(\frac{E-\epsilon_{0}}{2t}\right)-i\sqrt{1-\left(\frac{E+i0^{+}-\epsilon_{0}}{2t}\right)^{2}}\right]
$$


Estude a função de Green para $\left|E-\epsilon_{0}\right|\lessgtr2t$.

### Exercício 2.2. 
Implemente um código que resolva a relação auto-consistente de forma iterativa. Compare o resultado numérico com o resultado analítico. 
**Dica:** Adicione uma pequena parte imaginária positiva à energia para estabilizar o método iterativo, $E \rightarrow E + i \eta$

## 3. Cálculo dos estados propagantes

Vamos agora calcular os estados propagantes. Recordemos que estes são solução de:
$$
-\boldsymbol{v}^{\dagger}\lambda^{-1}\Psi_{n-1}+\left(E-\boldsymbol{h}\right)\Psi_{n}=\lambda\boldsymbol{v}\Psi_{n}
$$

Que pode ser transformada no problema de valores próprios generalizado:
$$
\left[\begin{array}{cc}
\boldsymbol{0} & \boldsymbol{1}\\
-\boldsymbol{v}^{\dagger} & \left(E-\boldsymbol{h}\right)
\end{array}\right]\left[\begin{array}{c}
\Psi_{n-1}\\
\Psi_{n}
\end{array}\right]=\lambda\left[\begin{array}{cc}
\boldsymbol{1} & \boldsymbol{0}\\
\boldsymbol{0} & \boldsymbol{v}
\end{array}\right]\left[\begin{array}{c}
\Psi_{n-1}\\
\Psi_{n}
\end{array}\right]
$$.

### Exercício 3.1.
Definida uma função que dados $\boldsymbol{h}$ e $\boldsymbol{v}$, construa o problema de valores próprios generalizado e o resolva. Compare os valores próprios obtidos com o resultado analítico.

### Exercício 3.2.
Recordando que os valores próprios, obtidos no exercício anterior, têm a forma:
$$
\left[\begin{array}{c}
\Psi_{n-1}\\
\Psi_{n}
\end{array}\right]
$$

Vamos escolher a seguinte normalização: $\Psi^{\dagger}\cdot\Psi=1$. 
Com esta normalização, calcule o operador de corrente:

$$
\left\langle \Psi_{\alpha}\left|I\right|\Psi_{\beta}\right\rangle =i\Psi_{\alpha}^{\dagger}\left(\lambda_{\beta}\boldsymbol{v}-\lambda_{\alpha}^{*}\boldsymbol{v}^{\dagger}\right)\Psi_{\beta}
$$

Verifique se a matriz é diagonal (ou diagonal por blocos). Compare as velocidades obtidas desta forma, com o resultado analítico.

## 4. Cálculo da auto-energia do contacto através de estados propagantes

Vamos agora calcular a auto-energia, usando o resultado:

$$
\boldsymbol{\Sigma}^R = \boldsymbol{v} \cdot \boldsymbol{\Psi}_{out} \cdot \boldsymbol{\Lambda}_{out} \cdot \boldsymbol{\Psi}_{out}^{-1}
$$

Onde `out` refere-se a estados com velocidade positiva, ou com $|\lambda|<1$, isto é estados que se propagam para dentro da lead e, logo, para fora da região central (outgoing states).

### Exercícío 4.1.
Tomado apenas os estados outgoing (com velocidade positiva, ou $|\lambda|<1$), use a equação anterior para calcular numericamente a auto-energia. Compare o resultado obtido, com o resultado analítico e o resultado obtido usando o método recursivo. 

## 5. Transporte usando a fórmula de Caroli

Consideremos o sistema de dois contactos que é decrito pelo Hamiltoninao:
$$
H_{2-\text{contactos}}=\left[\begin{array}{ccccc}
\boldsymbol{H}_{C} & \boldsymbol{V}_{C,L} & \boldsymbol{V}_{C,R}\\
\boldsymbol{V}_{L,C} & \boldsymbol{h}_{L} &  & \boldsymbol{v}_{L}\\
\boldsymbol{V}_{R,C} &  & \boldsymbol{h}_{R} &  & \boldsymbol{v}_{R}\\
 & \boldsymbol{v}_{L}^{\dagger} &  & \boldsymbol{H}_{L,\text{semi}} & \boldsymbol{v}_{R}\\
 &  & \boldsymbol{v}_{R}^{\dagger} & \boldsymbol{v}_{R}^{\dagger} & \boldsymbol{H}_{R,\text{semi}}
\end{array}\right]
$$

Onde incluímos um sítio de cada cadeia na região central. No formalismo de Caroli tal não é necessário, mas fazemo-lo para fazer a relação com o formalismo de Landauer-Büttiker.

A fórmula de Caroli escreve a transmitância como:

$$
\mathcal{T}(E)=\text{Tr}\left[\boldsymbol{G}^{A}(E)\boldsymbol{\Gamma}_{R}(E)\boldsymbol{G}^{R}(E)\boldsymbol{\Gamma}_{L}(E)\right]
$$

onde

$$
\boldsymbol{G}^{R}(E)=\left[\begin{array}{ccc}
E-\boldsymbol{H}_{C} & -\boldsymbol{V}_{C,L} & -\boldsymbol{V}_{C,R}\\
-\boldsymbol{V}_{L,C} & E-\boldsymbol{h}_{L}-\boldsymbol{\Sigma}_{L}^{R}(E)\\
-\boldsymbol{V}_{R,C} &  & E-\boldsymbol{h}_{R}-\boldsymbol{\Sigma}_{L}^{R}(E)
\end{array}\right]^{-1},
$$

$\boldsymbol{G}^{A}(E)=\left[\boldsymbol{G}^{R}(E)\right]^{\dagger}$ e

$$
\begin{align*}
\boldsymbol{\Gamma}_{L}(E) & =\left[\begin{array}{ccc}
0 & 0 & 0\\
0 & i\left(\boldsymbol{\Sigma}_{L}^{R}(E)-\boldsymbol{\Sigma}_{L}^{A}(E)\right) & 0\\
0 & 0 & 0
\end{array}\right]\\
\boldsymbol{\Gamma}_{R}(E) & =\left[\begin{array}{ccc}
0 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & i\left(\boldsymbol{\Sigma}_{R}^{R}(E)-\boldsymbol{\Sigma}_{R}^{A}(E)\right)
\end{array}\right]
\end{align*}
$$


Para propósitos numéricos, podemos escrever
$$
\mathcal{T}(E)=\text{Tr}\left[\boldsymbol{Q}_{L}^{\dagger}(E)\cdot\boldsymbol{Q}_{L}(E)\right]
$$

onde
$$
\begin{align*}
\boldsymbol{Q}_{L}(E) & =\left[\begin{array}{ccc}
E-\boldsymbol{H}_{C} & -\boldsymbol{V}_{C,L} & -\boldsymbol{V}_{C,R}\\
-\boldsymbol{V}_{L,C} & E-\boldsymbol{h}_{L}-\boldsymbol{\Sigma}_{L}^{R}(E)\\
-\boldsymbol{V}_{R,C} &  & E-\boldsymbol{h}_{R}-\boldsymbol{\Sigma}_{L}^{R}(E)
\end{array}\right]^{-1}\left[\begin{array}{c}
0\\
i\left(\boldsymbol{\Sigma}_{L}^{R}(E)-\boldsymbol{\Sigma}_{L}^{A}(E)\right)\\
0
\end{array}\right]\\
\boldsymbol{Q}_{R}(E) & =\left[\begin{array}{ccc}
E-\boldsymbol{H}_{C} & -\boldsymbol{V}_{C,L} & -\boldsymbol{V}_{C,R}\\
-\boldsymbol{V}_{L,C} & E-\boldsymbol{h}_{L}-\boldsymbol{\Sigma}_{L}^{R}(E)\\
-\boldsymbol{V}_{R,C} &  & E-\boldsymbol{h}_{R}-\boldsymbol{\Sigma}_{L}^{R}(E)
\end{array}\right]^{-1}\left[\begin{array}{c}
0\\
0\\
i\left(\boldsymbol{\Sigma}_{R}^{R}(E)-\boldsymbol{\Sigma}_{R}^{A}(E)\right)
\end{array}\right]
\end{align*}
$$

### Exercício 5.1.
Implemente as equações anteriores de forma a calcular a transmitância. 

### Exercício 5.2.
Compare a tarnsmitância calculada anteriormente com o transmitância obtida usando a fórmula de Landauer:

$$
\mathcal{T}(E)=\text{Tr}\left[\tilde{\boldsymbol{t}}_{R\leftarrow L}^{\dagger}\cdot\tilde{\boldsymbol{t}}_{R\leftarrow L}\right]
$$
onde
$$
\tilde{\boldsymbol{t}}_{R\leftarrow L}=i\boldsymbol{V}_{R}^{-1/2}\cdot\boldsymbol{\Psi}_{R,\text{out},p}^{\dagger}\cdot\boldsymbol{\Gamma}_{R}\cdot\boldsymbol{G}^{R}\cdot\boldsymbol{\Gamma}_{L}\cdot\boldsymbol{\Psi}_{L,\text{in},p}^{\dagger}\cdot\boldsymbol{V}_{L}^{-1/2}
$$

e $\boldsymbol{V}_{L/R}$ são matrizes diagonais das velocidades.